INF8770 Technologies multimédias

Polytechnique Montréal

Importation des librairies

In [38]:
from PIL import Image
import os
from skimage.feature.texture import local_binary_pattern
import glob
import cv2
import numpy
import csv

 Matrices de Co-occurrence (MC)

In [ ]:
def desc_MC(imagePath, dx, dy, color1, color2):
  image = Image.open(imagePath)
  width = len(image)
  height = len(image[0])
  result = []

  for column in range(0, width):
    if column + dx >= width:
      break
    for row in range(0, height):
      if row + dy >= height: 
        break
      areCorrespondingColors = True
      for canal in range(0, 3):
        if image[column][row][canal] != color1[canal] or image[column + dx][row + dy][canal] != color2[canal]:
           areCorrespondingColors = False
           break
      result += 1 if areCorrespondingColors else 0

  return result

Modèles Binaires Locaux (LBP)

In [ ]:
longueurCodes = 8
rayon = 1

def desc_LBP(path):
  image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
  desc =  local_binary_pattern(image, longueurCodes, rayon)
  return numpy.asarray(desc, numpy.float32)

Importation des images

In [ ]:
query = glob.glob(os.path.join("./data", "*.jpg"))
database = glob.glob(os.path.join("./data/database", "*.jpg"))

Méthode de comparaison

In [34]:
def intersection(vector_i, vector_j):
    return numpy.sum(numpy.minimum(vector_i, vector_j)) / numpy.sum(vector_j)

def norms1(vector_i, vector_j):
    return numpy.sum(numpy.abs(vector_i - vector_j))

def norms2(vector_i, vector_j):
    return numpy.sqrt(numpy.sum(numpy.power(vector_i - vector_j, 2)))

def bhattacharyya(vector_i, vector_j):
    return -numpy.log((numpy.sum(numpy.sqrt(numpy.multiply(vector_i, vector_j)))))

def MDPA(vector_i, vector_j):
    sum_m = 0
    for m in range(len(vector_i)):
        sum_n = 0
        for n in range(m):
            sum_n += vector_i[n] - vector_j[n]
        sum_m += abs(sum_n)

    return sum_m

def cosineSimilarity(vector_i, vector_j):
    return numpy.sum(numpy.multiply(vector_i, vector_j)) / (numpy.sqrt(numpy.sum(vector_i)) * numpy.sqrt(numpy.sum(vector_j)))

def hammingDistance(vector_i, vector_j):
    sum = 0
    for k in range(len(vector_i)):
        sum += 1 if (vector_i[k] == vector_j[k]) else 0 

    return sum

Modification des données

In [42]:


# TODO Faire les carrés englobants

for path_query in query:
    result_mc = dict()
    result_lbp = dict()

    #mc_query = desc_MC()
    lbp_query = desc_LBP(path_query)

    #row_mc, col_mc = cv2.imread(path_query, cv2.IMREAD_GRAYSCALE).shape
    row_lbp, col_lbp = cv2.imread(path_query, cv2.IMREAD_GRAYSCALE).shape

    #mc_hist_query, bins = numpy.histogram(mc_query, bins = range(256))
    lbp_hist_query, bins = numpy.histogram(lbp_query, bins = range(256))

    #mc_hist_query = mc_hist_query / (row_mc * col_mc)
    lbp_hist_query = lbp_hist_query / (row_lbp * col_lbp)

    methods = ["intersection", "norms1", "norms2", "bhattacharyya", "MDPA", "cosineSimilarity", "hammingDistance"]

    with open(f"./results/{path_query[7:-4]}.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        field = [""] + [x[16:-4] for x in database]
        writer.writerow(field)
        
        for key in methods:
            #result_mc[key] = []
            result_lbp[key] = []

        for path_db in database:
            # mc_db = desc_MC()
            lbp_db = desc_LBP(path_db)
            
            #row_mc, col_mc = cv2.imread(path_db, cv2.IMREAD_GRAYSCALE).shape
            row_lbp, col_lbp = cv2.imread(path_db, cv2.IMREAD_GRAYSCALE).shape

            #mc_hist_db, bins = numpy.histogram(mc_db, bins = range(256))
            lbp_hist_db, bins = numpy.histogram(lbp_db, bins = range(256))

            #mc_hist_db = mc_hist_db / (row_mc * col_mc)
            lbp_hist_db = lbp_hist_db / (row_lbp * col_lbp)

            result_id = path_query[7:-4] + ":" + path_db[16:-4]

            #result_mc["intersection"].append((result_id, intersection(mc_hist_query, mc_hist_db)))
            result_lbp["intersection"].append((result_id, intersection(lbp_hist_query, lbp_hist_db)))

            #result_mc["norms1"].append((result_id, norms1(mc_hist_query, mc_hist_db)))
            result_lbp["norms1"].append((result_id, norms1(lbp_hist_query, lbp_hist_db)))

            #result_mc["norms2"].append((result_id, norms2(mc_hist_query, mc_hist_db)))
            result_lbp["norms2"].append((result_id, norms2(lbp_hist_query, lbp_hist_db)))

            #result_mc["bhattacharyya"].append((result_id, bhattacharyya(mc_hist_query, mc_hist_db)))
            result_lbp["bhattacharyya"].append((result_id, bhattacharyya(lbp_hist_query, lbp_hist_db)))

            #result_mc["MDPA"].append((result_id, MDPA(mc_hist_query, mc_hist_db)))
            result_lbp["MDPA"].append((result_id, MDPA(lbp_hist_query, lbp_hist_db)))

            #result_mc["cosineSimilarity"].append((result_id, cosineSimilarity(mc_hist_query, mc_hist_db)))
            result_lbp["cosineSimilarity"].append((result_id, cosineSimilarity(lbp_hist_query, lbp_hist_db)))

            #result_mc["hammingDistance"].append((result_id, hammingDistance(mc_hist_query, mc_hist_db)))
            result_lbp["hammingDistance"].append((result_id, hammingDistance(lbp_hist_query, lbp_hist_db)))
            
        for key in result_lbp.keys():
            reverse = True if (key in ["intersection", "hammingDistance"]) else False
            print(key, sorted(result_lbp[key], key=lambda tp: tp[1], reverse=reverse)[:3])
            writer.writerow([key] + [result_lbp[key][1]])
    break


intersection [('airplane_query:airplane_4', 0.9285349455986982), ('airplane_query:airplane_3', 0.9098495553384187), ('airplane_query:airplane_1', 0.9074429863434066)]


TypeError: can only concatenate list (not "tuple") to list